In [1]:
from qutip import *
import numpy as np
import matplotlib.pyplot as plt
from qutip.measurement import measure, measurement_statistics
from qutip.qip.operations import snot
from qutip.qip.operations import rx
from qutip.qip.operations import ry
from qutip.qip.operations import rz
from scipy.optimize import curve_fit
import datetime
import qutip.control.pulseoptim as cpo
from qutip.qip.operations import cnot
import matplotlib as mpl
from matplotlib import cm
from math import *

In [57]:
#recreating figure 5b

#FT encoding - assume state is in logical minus
#states
z=basis(2,0) #|0>
n=basis(2,1) #|1>
p=(z+n).unit() #|+>
m=(z-n).unit() #|->

#gates
H=Qobj(np.array([[1/np.sqrt(2),1/np.sqrt(2)], [1/np.sqrt(2),-1/np.sqrt(2)]]))
X=sigmax()
Y=sigmay()
Z=sigmaz()
I=qeye(2)

#include flag qubit as first in tensor - will be 0 since FT encoding worked
minL = (tensor(z,z,z,z,z,z)-tensor(z,n,n,n,n,n)).unit()
#print(minL)

#first demonstrate that an errorless run of the stabilizer measurements leaves state unchanged
#measure A
#pauli error in 4th data qubit
state = (tensor(z,z,z,z,n,z)-tensor(z,n,n,n,z,n)).unit()

XXYIY = tensor(I,X,X,Y,I,Y)
s1ket = (XXYIY*minL).dag()
mA = (s1ket*state)*minL+(minL.dag()*state)*minL
print(mA)
#corrects the pauli error (other s_i correct other ones)

#include other s_is to account for all possible single qubit errors
XXYIY = tensor(I,X,X,Y,I,Y)
YXXYI = tensor(I,Y,X,X,Y,I)
IYXXY = tensor(I,I,Y,X,X,Y)
YIYXX = tensor(I,Y,I,Y,X,X)
s1ket = (XXYIY*minL).dag()
s2ket = (YXXYI*minL).dag()
s3ket = (IYXXY*minL).dag()
s4ket = (YIYXX*minL).dag()
s1=(s1ket*state)
s2=(s2ket*state)
s3=(s3ket*state)
s4=(s4ket*state)
print(s1,s2,s3,s4)
mA = (minL.dag()*state)*minL+s1*minL+s2*minL+s3*minL+s4*minL
print(mA)

#measure F
XX = tensor(X*X,I,I,I,I,I)
mF = XX*minL
print(isequal(minL,mF))

#now introduce the Y error on the aux qubit as in 5a
#X error on flag, Y3Y5 error
auxY = tensor(X,I,I,Y,I,Y)
errState = auxY*minL

#measure A
s1=(s1ket*errState)
s2=(s2ket*errState)
s3=(s3ket*errState)
s4=(s4ket*errState)
mA = (minL.dag()*errState)*minL+s1*minL+s2*minL+s3*minL+s4*minL
print(mA)
#this correction scheme doesn't work for two qubit error that is generated from the aux Y error!
#...without the flag qubit

#measure F
fmeas = XX*tensor(X,I,I,I,I,I)*minL
print(isequal(fmeas,minL))
#flag is now raised!
#knowing the flag is raised and what s1-s4 are allows us to detect the error

Quantum object: dims = [[2, 2, 2, 2, 2, 2], [1, 1, 1, 1, 1, 1]], shape = (64, 1), type = ket
Qobj data =
[[ 0.70710678]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [-0.70710678]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.     

In [84]:
#try all possible single and two qubit Pauli errors and determine which ones can be
#corrected by this scheme
minL = (tensor(z,z,z,z,z,z)-tensor(z,n,n,n,n,n)).unit()
XXYIY = tensor(I,X,X,Y,I,Y)
YXXYI = tensor(I,Y,X,X,Y,I)
IYXXY = tensor(I,I,Y,X,X,Y)
YIYXX = tensor(I,Y,I,Y,X,X)

s1ket = (XXYIY*minL).dag()
s2ket = (YXXYI*minL).dag()
s3ket = (IYXXY*minL).dag()
s4ket = (YIYXX*minL).dag()

#all possible errors
#single qubit - no error on flag qubit
aX1 = tensor(I,X,I,I,I,I)
aX2 = tensor(I,I,X,I,I,I)
aX3 = tensor(I,I,I,X,I,I)
aX4 = tensor(I,I,I,I,X,I)
aX5 = tensor(I,I,I,I,I,X)

aY1 = tensor(I,Y,I,I,I,I)
aY2 = tensor(I,I,Y,I,I,I)
aY3 = tensor(I,I,I,Y,I,I)
aY4 = tensor(I,I,I,I,Y,I)
aY5 = tensor(I,I,I,I,I,Y)

aZ1 = tensor(I,Z,I,I,I,I)
aZ2 = tensor(I,I,Z,I,I,I)
aZ3 = tensor(I,I,I,Z,I,I)
aZ4 = tensor(I,I,I,I,Z,I)
aZ5 = tensor(I,I,I,I,I,Z)
sqerrs = [aX1,aX2,aX3,aX4,aX5,aY1,aY2,aY3,aY4,aY5,aZ1,aZ2,aZ3,aZ4,aZ5]


def calc_stabs(a):
    errState = a*minL
    #print(errState)

    #measure A
    #print(s3ket.dag())
    s1=(s1ket*errState)
    s2=(s2ket*errState)
    s3=(s3ket*errState)
    s4=(s4ket*errState)
    
    
    
    #correction attempt without flag
    mA = (minL.dag()*errState)*minL+s1*minL+s2*minL+s3*minL+s4*minL
    print(isequal(mA,minL))
    return [s1,s2,s3,s4]

for i in range(10,15):
    print(calc_stabs(sqerrs[i]))
    
def calc_flag(a):
    #measure F
    fmeas = XX*tensor(X,I,I,I,I,I)*minL
    print(isequal(fmeas,minL))

False
[Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]], Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]], Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]], Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]]
False
[Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]], Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]], Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]], Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]]]
False
[Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]], Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]], Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.]], Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0